# Contents
 - [Imports](#Imports)
 - [Functions](#Functions)
 - [Make Surface & Visualize](#Surface)
 - [Make Ethane & Visualize](#Ethane)
 - [Apply Forcefields and Generate Simulation Box](#Forcefields)
 - [Define Statepoints](#Statepoints)
 - [Render Trajectory](#Trajectory)
 - [Errors](#Errors)
 - [Notes](#Notes)

# Interesting Etherpads
 - [7-9: Equilibration & Box Stuff](https://etherpad.boisestate.edu/p/ldrd-s20-9)
 - [7-30: New Sweeps](https://etherpad.boisestate.edu/p/ldrd-s20-12)

In [1]:
# Quick commands
#!tail -n 1 signac/workspace/*/surfequil*.prp
#!head -n 10 signac/workspace/*/surfprod*.prp
!rm -rf *.in.* species* *.out* *.inp*

<a id="Imports"></a>
## Imports

In [2]:
import mosdef_cassandra as mc
import generate_mc as gen   # Chris' script
import mbuild as mb
import foyer
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import random
import unyt as u    # For Cassandra's physical unit specification 
                    # https://mosdef-cassandra.readthedocs.io/en/latest/guides/unyts.html

<a id="Functions"></a>
## Functions

In [3]:
def gen_system(dims=[2, 2, 2]):
    defaults_dict = {'stoichiometry': {'Mo': 1, 'V': 0.15, 'Nb': 0.13, 'Te': 0.12},
                     'dimensions': dims,
                     'template': 'M1UnitCell_mod.pdb',
                     'crystal_x': 2.148490,
                     'crystal_y': 2.664721,
                     'crystal_z': 0.400321,
                     'z_reactor_size': 20.0,
                     'forcefield': None
                     }

    system = gen.Surface(defaults_dict['dimensions'],
                        defaults_dict['template'],
                         defaults_dict['stoichiometry'],
                         True,
                         defaults_dict['crystal_x'],
                         defaults_dict['crystal_y'],
                         defaults_dict['crystal_z']
                        )
    system.translate_to(np.zeros(3)) # Added so that periodicity doesn't error out
    system.periodicity = system.boundingbox.lengths # necessary
    #system.to_parmed() # Triggers an error
    return system

In [4]:
def apply_ff(system, systemFF, organicFF=None):
    typed_surface = systemFF.apply(system.children[0],
                            assert_angle_params=False,
                            assert_bond_params=False,
                            assert_dihedral_params=False,
                            assert_improper_params=False)
    if organicFF:
        for idx, child in enumerate(system.children):
            if idx == 0:
                pass
            else:
                typed_comp = organicFF.apply(child)
                typed_surface += typed_comp
    return typed_surface

In [10]:
def run_cassandra_surf(chem_pot, temp):
    # Create box and species list
    box_list = [surface]
    species_list = [typed_surface,typed_ethane]

    # Since we have an occupied box we need to specify
    # the number of each species present in the intial config
    mols_in_boxes = [[2,0]]

    system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
    moveset = mc.MoveSet("gcmc", species_list)

    custom_args = {
        "run_name": f"surfequil_{chem_pot:.0f}_{temp:.0f}",
        "chemical_potentials": ["none",chem_pot*u.Unit('kJ/mol')],
        "rcut_min": 0.3980 * 2.5* u.angstrom, #(or 3.0)
        "vdw_cutoff": min(box.lengths)/2.1* u.angstrom,
        "charge_style": "none",
        #"charge_cutoff": 14.0,
        "coord_freq": 100,
        "prop_freq": 10,
    }

    mc.run(
        system=system, 
        moveset=moveset, 
        run_type="equilibration", 
        run_length= 10000, # To reach ~1.33 hours
        temperature=temp*u.K, 
        **custom_args
    )
    
    # Set max translate and volume for production
    moveset.max_translate = [[0* u.angstrom,10.0* u.angstrom]] # angstroms

    # Update run_name and restart_name
    custom_args["run_name"] = f"surfprod_{chem_pot:.0f}_{temp:.0f}"
    custom_args["restart_name"] = f"surfequil_{chem_pot:.0f}_{temp:.0f}"

    mc.restart(
        system=system,
        moveset=moveset,
        run_type="production",
        run_length= 50000, # To reach ~6.67 hours, total 8 hrs
        temperature=temp*u.K,
        **custom_args,
    )

In [6]:
mc.utils.detect_cassandra_binaries();

Using the following executables for Cassandra:
Python: /home/nealeellyson/miniconda3/envs/moscas/bin/python
library_setup: /home/nealeellyson/miniconda3/envs/moscas/bin/library_setup.py
Cassandra: /home/nealeellyson/miniconda3/envs/moscas/bin/cassandra.exe


<a id="Surface"></a>
## Make Surface & Visualize

In [7]:
surface = gen_system(dims=[1, 1, 2]) 
surface.periodicity = surface.periodicity + np.array([0,0,1.4-0.85032099]) # So that z = 1.4Angstrom
surface.translate_to([0,0,0]) # Centering
surface.visualize(show_ports=True, color_scheme={'O' : 'red',
                                                 'Te': 'black',
                                                 'V' : 'black',
                                                 'Nb': 'black',
                                                 'Mo': 'black'})

Adding [0, 0, 1] to system... 


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<a id="Ethane"></a>
## Make Ethane & Visualize

In [8]:
ethane = mb.load("CC", smiles=True)
ethane.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<a id="Forcefields"></a>
## Apply Forcefields and Generate Simulation Box

In [9]:
# Load forcefields
opls_uff = foyer.forcefields.Forcefield(forcefield_files='forcefields/FF_opls_uff.xml')
oplsaa = foyer.forcefields.load_OPLSAA()

# Use foyer to apply forcefields
typed_surface = apply_ff(surface, opls_uff)
typed_ethane = oplsaa.apply(ethane, 
                            assert_bond_params=False, 
                            assert_angle_params=False, 
                            assert_dihedral_params=False)

# Establish simulation box dimensions
L = 10
box = mb.Box(mins=[-L/2]*3, maxs=[L/2]*3)

/home/nealeellyson/miniconda3/envs/moscas/lib/python3.7/site-packages/foyer/forcefield.py:449: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/home/nealeellyson/miniconda3/envs/moscas/lib/python3.7/site-packages/foyer/forcefield.py:461: UserWarning: No force field name found in force field XML file.
  'No force field name found in force field XML file.'
/home/nealeellyson/miniconda3/envs/moscas/lib/python3.7/site-packages/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)
/home/nealeellyson/miniconda3/envs/moscas/lib/python3.7/site-packages/parmed/openmm/topsystem.py:238: OpenMMWarning: Adding what seems to be Urey-Bradley terms before Angles. This is unexpected, but the parameters will all be present in one form or another.
  'all be present in one form or another.', OpenMMWarning)
/home/nealeellyson/mini

<a id="Statepoints"></a>
## Define Statepoints

In [11]:
eps = typed_ethane.atoms[0].epsilon
print(f"epsilon: {eps:.2f}")
print("sigma: ", typed_ethane.atoms[0].sigma)
# ParmEd is in kcal/mol

kb = 0.001985875    # kcal/(mol K)
T = eps/kb

print(f"The unit temperature is {T:.0f} K")
chem_pots = [i*T for i in range(0,5)] 
temp = 2*T

epsilon: 0.07
sigma:  3.5
The unit temperature is 33 K


<a id="Run"></a>
## Run Simulation

In [ ]:
for chem_pot in chem_pots:
    run_cassandra_surf(chem_pot,temp)

Treating <Structure 160 atoms; 1 residues; 0 bonds; PBC (orthogonal); NOT parametrized> as a non-insertable rigid species since it has no bonds
Using the following executables for Cassandra:
Python: /home/nealeellyson/miniconda3/envs/moscas/bin/python
library_setup: /home/nealeellyson/miniconda3/envs/moscas/bin/library_setup.py
Cassandra: /home/nealeellyson/miniconda3/envs/moscas/bin/cassandra.exe
Generating fragment libraries...


/home/nealeellyson/miniconda3/envs/moscas/lib/python3.7/site-packages/mbuild/formats/cassandramcf.py:178: UserWarning: No bonds found. Cassandra will interpet this as a rigid species
  warnings.warn("No bonds found. Cassandra will interpet "
/home/nealeellyson/miniconda3/envs/moscas/lib/python3.7/site-packages/mbuild/formats/cassandramcf.py:570: UserWarning: More than two atoms present but no fragments identified.
  warnings.warn('More than two atoms present but '


Running Cassandra...
Using the following executables for Cassandra:
Python: /home/nealeellyson/miniconda3/envs/moscas/bin/python
library_setup: /home/nealeellyson/miniconda3/envs/moscas/bin/library_setup.py
Cassandra: /home/nealeellyson/miniconda3/envs/moscas/bin/cassandra.exe
Running Cassandra...
Treating <Structure 160 atoms; 1 residues; 0 bonds; PBC (orthogonal); NOT parametrized> as a non-insertable rigid species since it has no bonds
Using the following executables for Cassandra:
Python: /home/nealeellyson/miniconda3/envs/moscas/bin/python
library_setup: /home/nealeellyson/miniconda3/envs/moscas/bin/library_setup.py
Cassandra: /home/nealeellyson/miniconda3/envs/moscas/bin/cassandra.exe
Generating fragment libraries...
Running Cassandra...
Using the following executables for Cassandra:
Python: /home/nealeellyson/miniconda3/envs/moscas/bin/python
library_setup: /home/nealeellyson/miniconda3/envs/moscas/bin/library_setup.py
Cassandra: /home/nealeellyson/miniconda3/envs/moscas/bin/cas

<a id="Trajectory"></a>
## Render Trajectory

In [27]:
import ex_render_cassandra as erc
import os
import getpass

# These cells are for visualizing output
xyzfile = f"prod_{chem_pot:.0f}_{temp:.0f}.out.xyz"
boxfile = f"prod_{chem_pot:.0f}_{temp:.0f}.out.H"
traj = erc.make_traj(xyzfile, boxfile)
print("Frames: ",len(traj))

cwd = os.getcwd()
username = getpass.getuser()
for chem_pot in chem_pots:
    xyzfile = f"surfprod_{chem_pot:.0f}_{temp:.0f}.out.xyz"
    print("\n\tCopy this into Ovito Load Remote File:") # From Fry
    print('sftp://'+username+'@fry'+cwd+'/'+xyzfile)

Copy this into Ovito Load Remote File:
/home/nealeellyson/CassandraSim/output/gcmc.out.xyz


In [26]:
xyzfile = "output/gcmc.out.xyz"
boxfile = "output/gcmc.out.H"

traj = erc.make_traj(xyzfile, boxfile)

In [27]:
erc.display_movie(erc.render_sphere_frame, traj)
#erc.display_movie(erc.render_sphere_frame, traj, gif="traj.gif")

<a id="Errors"></a>
## Errors:
`Initial vdw cutoff is greater than half the minimum box length
This error occurred in subroutine Get_Pair_Style on step 0.`

Jenny: 
> Fixed by making the boxlist an empty box with correct size

My fix:
```
"vdw_cutoff": min(surface.boundingbox.lengths)/3,
"charge_style": "none",
#"charge_cutoff": 14.0,
```


<a id="Notes"></a>
## Notes

#### W/rt equilibration
We can tweak trial move stuff within Cassandra and could dramatically improve equilibration times

#### W/rt box size
We want a surface with ethane absorbing into it and the box height above to only focus on things absorbing into the surface. The problem is the other side of the surface and if the space is too small where the molecules can't fit between the two. But if we make it too far then we have a middle area. 